# Layers and Blocks

To begin, we revisit the code
that we used to implement MLPs

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.0733, -0.1016, -0.1408,  0.0874,  0.0808,  0.0452,  0.1052,  0.1437,
          0.0462,  0.0033],
        [-0.1402, -0.0144, -0.0131,  0.0647, -0.0310,  0.0405,  0.2073, -0.0270,
         -0.0483,  0.1304]], grad_fn=<AddmmBackward>)

`nn.Sequential` defines a special kind of `Module`

A Custom Block

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

Instantiate the MLP's layers
and subsequently invoke these layers

In [3]:
net = MLP()
net(X)

tensor([[-0.0505, -0.1906,  0.0944, -0.0260,  0.0331,  0.4239, -0.0627,  0.1947,
         -0.1135, -0.1321],
        [-0.0587, -0.1707,  0.1580, -0.0722, -0.0933,  0.1187, -0.1583,  0.2753,
         -0.1189,  0.0157]], grad_fn=<AddmmBackward>)

The Sequential Block

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-5.5843e-02, -6.8424e-02, -2.8254e-01, -9.8593e-02,  2.4024e-02,
          2.2788e-01, -9.6547e-02, -9.5748e-02,  1.1019e-01,  1.2251e-04],
        [ 5.9271e-02, -6.0811e-02, -1.2770e-01, -1.0038e-01,  2.7286e-02,
          9.5549e-02, -1.9524e-02, -1.3680e-01,  7.8278e-02,  6.7509e-02]],
       grad_fn=<AddmmBackward>)

Executing Code in the Forward Propagation Function

In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(0.2618, grad_fn=<SumBackward0>)

Mix and match various
ways of assembling blocks together

In [8]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.2828, grad_fn=<SumBackward0>)